In [1]:
import sqlalchemy
import pandas as pd
from IPython.display import HTML


from cg_analysis import (
    DB_URL,
    CG_COL_TYPES,
    CG_DATETYPE_COLS,
    LI_COL_TYPES,
    PT_COL_TYPES,
    PT_DATETYPE_COLS,
    FIN_COL_TYPES,
    FIN_DATETYPE_COLS,
    FIN_ADOPTION_COL_TYPES,
    DATA_PATH,
    OUTPUT_CG_PATH,
)
from cg_analysis import cg_query, li_query, pt_query, fin_query

In [2]:
import importlib
import cg_analysis

importlib.reload(cg_analysis)

<module 'cg_analysis' from '/home/robyn/repositories/data-science/investigations/one-switch/cg_analysis.py'>

In [3]:
engine = sqlalchemy.create_engine(DB_URL)

# load carton grouping data
cg_df = pd.read_sql_query(
    sql=cg_query, con=engine, dtype=CG_COL_TYPES, parse_dates=CG_DATETYPE_COLS
)

# load line items data
li_df = pd.read_sql_query(sql=li_query, con=engine, dtype=LI_COL_TYPES)

# load pallet timeline data
pt_df = pd.read_sql_query(
    sql=pt_query,
    con=engine,
    dtype=PT_COL_TYPES,
    parse_dates=PT_DATETYPE_COLS,
)

# load finance data
fin_df = pd.read_sql_query(
    sql=fin_query,
    con=engine,
    dtype=FIN_COL_TYPES,
    parse_dates=FIN_DATETYPE_COLS,
)

# fin_adoption_df = pd.read_sql_query(
#     sql=fin_adoption_query,
#     con=engine,
#     dtype=FIN_ADOPTION_COL_TYPES,
# )
fin_adoption_df = pd.read_csv(
    DATA_PATH / "fin_adoption.csv",
    dtype=FIN_ADOPTION_COL_TYPES,
)

In [4]:
# Add std_cartons data

cg_df = cg_df.merge(
    pt_df[["cg_id", "std_cartons"]], left_on="id", right_on="cg_id", how="left"
)

# Data completeness

Calculate feature prevalence and by extension, the availability of data

Look into finance data completeness

- % cgs with finance data over time (for all commodities)
- % cgs with inferred finance data from orders


In [5]:
# Total carton groupings by data availability
cg_ids = set(cg_df["id"])

stats_df = pd.DataFrame(
    {
        "Category": [
            "Total Carton Groupings",
            "Orders Data Only",
            "Finance Data Only",
            "Both",
            "Neither",
        ],
        "Count": [
            len(cg_ids),
            len(
                set(
                    cg_df[
                        (cg_df["line_item_id"].notna())
                        & (~cg_df["id"].isin(fin_df["cg_id"]))
                    ]["id"]
                )
            ),
            len(
                set(
                    cg_df[
                        (cg_df["line_item_id"].isna())
                        & (cg_df["id"].isin(fin_df["cg_id"]))
                    ]["id"]
                )
            ),
            len(
                set(
                    cg_df[
                        (cg_df["line_item_id"].notna())
                        & (cg_df["id"].isin(fin_df["cg_id"]))
                    ]["id"]
                )
            ),
            len(
                set(
                    cg_df[
                        (cg_df["line_item_id"].isna())
                        & (~cg_df["id"].isin(fin_df["cg_id"]))
                    ]["id"]
                )
            ),
        ],
    }
)

# Add proportion column
stats_df["Proportion"] = (stats_df["Count"] / len(cg_ids) * 100).round(2)

# Display the DataFrame
print("\nTotal carton groupings by data availability")
display(stats_df)


Total carton groupings by data availability


,Category,Count,Proportion
0,Total Carton Groupings,826688,100.00
1,Orders Data Only,328496,39.74
2,Finance Data Only,141,0.02
3,Both,67777,8.20
4,Neither,430274,52.05


# Heatmaps


In [212]:
from matplotlib import pyplot as plt
from cg_analysis import create_heatmap, create_heatmap_packing_week, save_figures_to_pdf

plt.ioff()  # Turn off interactive mode


# Heatmap 1: Commodity by local market
hm_local_1, pivot_local_1 = create_heatmap(
    cg_df,
    "commodity_name",
    "local_market",
    "container_number",
    "Heatmap of Commodity by Local Market (Container Count)",
    normalize_rows=True,
)
# display(HTML(pivot_local_1.to_html()))

# Heatmap 2: Variety by local market
hm_local_2, pivot_local_2 = create_heatmap(
    cg_df,
    "variety_name",
    "local_market",
    "container_number",
    "Heatmap of Variety by Local Market (Container Count)",
    normalize_rows=True,
)

# Heatmap 3: Production region by local market
hm_local_3, pivot_local_3 = create_heatmap(
    cg_df,
    "production_region",
    "local_market",
    "container_number",
    "Heatmap of Production Region by Local Market (Container Count)",
    normalize_rows=True,
)

# Heatmap 4: Seller by local market
hm_local_4, pivot_local_4 = create_heatmap(
    cg_df,
    "seller_id",
    "local_market",
    "container_number",
    "Heatmap of Seller by Local Market (Container Count)",
    normalize_rows=True,
)

# Heatmap 5: Local market by JBIN
hm_local_5, pivot_local_5 = create_heatmap(
    cg_df,
    "local_market",
    "jbin",
    "container_number",
    "Heatmap of Local Market by JBIN (Container Count)",
    normalize_rows=True,
)

# Heatmap 6: Local market by size count
hm_local_6, pivot_local_6 = create_heatmap(
    cg_df,
    "size_count",
    "local_market",
    "container_number",
    "Heatmap of Local Market by Size Count (Container Count)",
    normalize_rows=True,
)

# Heatmap 7: Packing week by local market
hm_local_7, pivot_local_7 = create_heatmap_packing_week(
    cg_df,
    "local_market",
    "container_number",
    title="Heatmap of Packing Week by Local Market (Container Count)",
    normalize_rows=True,
)

In [ ]:
local_market_heatmap_figures = [
    hm_local_1,
    hm_local_2,
    hm_local_3,
    hm_local_4,
    hm_local_5,
    hm_local_6,
    hm_local_7,
]
output_file = OUTPUT_CG_PATH / "local_market_heatmap_plots.pdf"
plt.ion()
save_figures_to_pdf(
    local_market_heatmap_figures, output_file, "Local Market Heatmap Plots"
)

## Local Market

### Local market vs commodity

- Soft Citrus and Pomelo have a statistically significant higher proportion of volumes going to Local Market. Is there an opportunity to push more of these commodities to international markets?
- Most limes go to Local Market (low volumes)

### Local market vs variety

- Navels (18%), Tango (17%) and Cambria Navels (8%) have a statistically significant higher proportion of volumes going to Local Market than average (5.5%). Is there an opportunity to push more of these commodities to international markets?
- Witkrans Late Navel (100%), Cambria Late Navels, Seedless Lemons, Clementines (99%), Valencias, Delta Seedless (98%), Lemons and Benny Valencia (97%) are almost exclusively sold to international markets.

### Local market vs production region

- Senwes (19%) and Hoedspruit (11%) are production regions that favour local markets (vs average of 6.4%).

### Local market vs sellers

- 8 sellers cater exclusively to the international market
- 2nd and 3rd top sellers (1190 and 699) are almost exclusively for the international market
- Sellers 345 (28%), 696 (14%), 695 (22%) and 246 (35%) have a statistically significant higher proportion of volumes going to the Local Market. Is there an opportunity to push more of these commodities to international markets? Are any of these clients with OneSwitch?

### Local market vs packing week

- Start and end of season (where there are lower volumes) have statistically significant higher proportions of volumes going to Local Market.
- Ramping up to peak season, W22 (9%), W23 (12%) and W24 (8%) have statistically significant higher proportions of volumes going to Local Market (average of ~5%)
- Peak season (W32, W33 and W34) is almost exclusively international markets (98%).

### Local market vs JBIN


In [214]:
# Generate heatmaps

plt.ioff()

# Heatmap 1: Commodity by target country
hm_country_1, pivot_country_1 = create_heatmap(
    cg_df,
    "commodity_name",
    "target_country",
    "container_number",
    "Heatmap of Commodity by Target Country (Container Count)",
    figsize=(20, 20),
    normalize_rows=True,
)
# display(HTML(hm_tm_1.to_html()))

# Heatmap 2: Variety by target country
# Filter out rows with missing target country and variety
hm_country_2, pivot_country_2 = create_heatmap(
    cg_df,
    "variety_name",
    "target_country",
    "container_number",
    "Heatmap of Variety by Target Country (Container Count)",
    figsize=(20, 20),
    normalize_rows=True,
)

# Heatmap 3: Production region by target country
hm_country_3, pivot_country_3 = create_heatmap(
    cg_df,
    "production_region",
    "target_country",
    "container_number",
    "Heatmap of Production Region by Target Country (Container Count)",
    figsize=(20, 20),
    normalize_rows=True,
)

# Heatmap 4: Seller by target country
hm_country_4, pivot_country_4 = create_heatmap(
    cg_df,
    "seller_id",
    "target_country",
    "container_number",
    "Heatmap of Seller by Target Country (Container Count)",
    figsize=(20, 20),
    normalize_rows=True,
)

# Heatmap 5: Target country by JBIN
hm_country_5, pivot_country_5 = create_heatmap(
    cg_df,
    "target_country",
    "jbin",
    "container_number",
    "Heatmap of Target Country by JBIN (Container Count)",
    figsize=(20, 20),
    normalize_rows=True,
)

# Heatmap 6: Size count by target country
hm_country_6, pivot_country_6 = create_heatmap(
    cg_df,
    "jbin",
    "target_country",
    "container_number",
    "Heatmap of Size Count by Target Country (Container Count)",
    figsize=(20, 20),
    normalize_rows=True,
)


# Heatmap 7: Packing week by target country
hm_country_7, pivot_country_7 = create_heatmap_packing_week(
    cg_df,
    "target_country",
    "container_number",
    title="Heatmap of Packing Week by Target Country (Container Count)",
    normalize_rows=True,
)

In [ ]:
target_country_heatmap_figures = [
    hm_country_1,
    hm_country_2,
    hm_country_3,
    hm_country_4,
    hm_country_5,
    hm_country_6,
    hm_country_7,
]
output_file = OUTPUT_CG_PATH / "target_country_heatmap_plots.pdf"
plt.ion()
save_figures_to_pdf(
    target_country_heatmap_figures, output_file, "Target Country Heatmap Plots"
)

## Target country

### Target country vs commodity

- Top pairs:
  - Orange-NL
  - Orange-SA
  - Soft Citrus-NL
  - Lemon-SA, Lemon-NL (about equal)
  - Orange-RU
- Notable pairs
  - Grapefruit-JP (JP is Grapefruit's 2nd largest market after NL)
  - Grapefruit-CN
  - Pomelo-RU and Pomelo-HK
  - NL top target country for major commodities, except for Lemons where Russia is top

### Target country vs variety

- Half of Cambria Navels are sent to SA
- Second largest market for Seedless lemons is CA
- NL and SA combined have > 50% market share for the following commodities
  - Eureka
  - Nardacott LS
  - Cambria Navels
  - Delta seedless
  - Leanri
  - Navelate
  - Benny valencia
  - Cambria late nabel
  - Witkrans Navel

### Target country vs seller

- NL and SA combined have > 50% market share for the following sellers
  - 1990 (84%)
  - 699 (73%)
  - 70 (50%)
  - 16 (82%)
  - 73 (58%)
- Sellers 246 and 345 are large contributors to local market

### Target country vs packing week

- SA is primary market from W01 - W12
- NL is primary market from W20 - W41, especially dominates in weeks W31 - W36 (highest volume weeks)
- RU has statistically significant higher representation in weeks W14 - W18 and W38 - W40
- CN has statistically significant higher representation in weeks W12 - W14 and W28 - W30
- AE has statistically significant higher representation in weeks W38 - W40
- HK has statistically significant higher representation in weeks W05 - W09


In [216]:
# Packing week by jbin
hm_pw_jbin, pivot_pw_jbin = create_heatmap_packing_week(
    cg_df,
    "jbin",
    "container_number",
    title="Heatmap of Packing Week by JBIN (Container Count)",
    normalize_rows=True,
)

# Packing week by commodity
hm_pw_comm, pivot_pw_comm = create_heatmap_packing_week(
    cg_df,
    "commodity_name",
    "container_number",
    title="Heatmap of Packing Week by Commodity (Container Count)",
    normalize_rows=True,
)

# Packing week by variety
hm_pw_var, pivot_pw_var = create_heatmap_packing_week(
    cg_df,
    "variety_name",
    "container_number",
    title="Heatmap of Packing Week by Variety (Container Count)",
    normalize_rows=True,
)

# Packing week by size count
hm_pw_size, pivot_pw_size = create_heatmap_packing_week(
    cg_df,
    "size_count",
    "container_number",
    title="Heatmap of Packing Week by Size Count (Container Count)",
    normalize_rows=True,
)

In [ ]:
pw_heatmap_figures = [
    hm_pw_jbin,
    hm_pw_comm,
    hm_pw_var,
    hm_pw_size,
]
output_file = OUTPUT_CG_PATH / "packing_week_heatmap_plots.pdf"
plt.ion()
save_figures_to_pdf(pw_heatmap_figures, output_file, "Packing Week Heatmap Plots")

## Packing week


# Market concentration analysis

The **Herfindahl-Hirschman Index (HHI)** is a measure of market concentration, commonly used in economics and competition analysis. It helps quantify how competitive or monopolistic a market is.

**Raw HHI**

The HHI is calculated as the **sum of the squares of the market shares** of all firms in the market:

$HHI = \sum_{i=1}^{N} s_i^2$

- $s_i$ is the market share of firm $i$ (expressed as a **decimal**, e.g., 0.4 for 40%)
- $N$ is the total number of firms
- Raw HHI ranges from $\frac{1}{N}$ (perfect competition) to 1 (monopoly)

**Normalised HHI**

The **Normalized HHI** adjusts the raw HHI to a fixed range between **0 and 1**, allowing comparisons between markets of different sizes (i.e., different numbers of firms).

$HHI_{normalized} = \frac{HHI - \frac{1}{N}}{1 - \frac{1}{N}}$

Where:

- $HHI$ = raw Herfindahl-Hirschman Index (using decimal shares, e.g., 0.4 for 40%)
- $N$ = number of firms in the market
- $\frac{1}{N}$ = minimum possible HHI when all firms are equal (perfect competition)
- `1` = maximum possible HHI (monopoly)


## Buyer concentration


In [218]:
from cg_analysis import (
    calculate_concentration_metrics,
    plot_concentration_bubble,
    save_figures_to_pdf,
)

plt.ioff()

commodity_buyer_concentration_metrics = calculate_concentration_metrics(
    cg_df, "commodity_name", "buyer_id"
)
fig1 = plot_concentration_bubble(
    commodity_buyer_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Buyer Market Concentration by Commodity",
)

variety_buyer_concentration_metrics = calculate_concentration_metrics(
    cg_df, "variety_name", "buyer_id"
)
fig2 = plot_concentration_bubble(
    variety_buyer_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Buyer Market Concentration by Variety",
)

production_region_buyer_concentration_metrics = calculate_concentration_metrics(
    cg_df, "production_region", "buyer_id"
)
fig3 = plot_concentration_bubble(
    production_region_buyer_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Buyer Market Concentration by Production Region",
)
cg_df_copy = cg_df.copy()
cg_df_copy["seller_id"] = cg_df_copy["seller_id"].astype(str)

seller_buyer_concentration_metrics = calculate_concentration_metrics(
    cg_df_copy, "seller_id", "buyer_id"
)
fig4 = plot_concentration_bubble(
    seller_buyer_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Buyer Market Concentration by Seller",
)

packing_week_buyer_concentration_metrics = calculate_concentration_metrics(
    cg_df, "packing_week", "buyer_id"
)
fig5 = plot_concentration_bubble(
    packing_week_buyer_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Buyer Market Concentration by Packing Week",
)

In [ ]:
buyer_concentration_figures = [fig1, fig2, fig3, fig4, fig5]
output_file = OUTPUT_CG_PATH / "buyer_concentration_plots.pdf"

plt.ion()
save_figures_to_pdf(
    buyer_concentration_figures, output_file, "Buyer Market Concentration Plots"
)

## Target country concentration


In [220]:
plt.ioff()

commodity_target_country_concentration_metrics = calculate_concentration_metrics(
    cg_df, "commodity_name", "target_country"
)
fig6 = plot_concentration_bubble(
    commodity_target_country_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Target Country Market Concentration by Commodity",
)

variety_target_country_concentration_metrics = calculate_concentration_metrics(
    cg_df, "variety_name", "target_country"
)
fig7 = plot_concentration_bubble(
    variety_target_country_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Target Country Market Concentration by Variety",
)

production_region_target_country_concentration_metrics = (
    calculate_concentration_metrics(cg_df, "production_region", "target_country")
)
fig8 = plot_concentration_bubble(
    production_region_target_country_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Target Country Market Concentration by Production Region",
)

seller_target_country_concentration_metrics = calculate_concentration_metrics(
    cg_df, "seller_id", "target_country"
)
fig9 = plot_concentration_bubble(
    seller_target_country_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Target Country Market Concentration by Seller",
)

packing_week_target_country_concentration_metrics = calculate_concentration_metrics(
    cg_df, "packing_week", "target_country"
)
fig10 = plot_concentration_bubble(
    packing_week_target_country_concentration_metrics,
    figsize=(12, 8),
    min_pallets=10,
    title="Target Country Market Concentration by Packing Week",
)

In [ ]:
target_country_concentration_figures = [fig6, fig7, fig8, fig9, fig10]
output_file = OUTPUT_CG_PATH / "target_country_concentration_plots.pdf"
plt.ion()
save_figures_to_pdf(
    target_country_concentration_figures,
    output_file,
    title="Target Country Market Concentration Plots",
)